In [1]:
import pandas as pd
import numpy as np

- Everything below the mean -> 1 : moderately useful
- Everything above the mean and below 2*mean : useful
- Everything above 2*the mean : super useful

In [4]:
from datasets import load_dataset

dataset = load_dataset('../data/ReviewPrediction', data_files={'train': 'train.csv', 'test': 'test.csv', 'validation': 'validation.csv'})


/home/liamt/Uni/Master/HS23/EITASP/Essentials-in-Text-and-Speech-Processing/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from transformers import DistilBertTokenizer
import torch

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-german-cased")

In [20]:
def tokenize_function(example):
    return tokenizer(example['review'], truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

In [22]:
tokenized_dataset['train']

Dataset({
    features: ['Unnamed: 0', '_id', 'review', 'score', 'upvotes', 'downvotes', 'sum', 'input_ids', 'attention_mask'],
    num_rows: 7656
})

In [8]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [27]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [31]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [34]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-german-cased", num_labels=4
)

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-german-cased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,attention_mask.